<a href="https://colab.research.google.com/github/GIM494/Learning-Design-Rules/blob/master/T5_Copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x
!pip install transformers
from transformers import T5Tokenizer

     |████████████████████████████████| 890kB 3.4MB/s 
     |████████████████████████████████| 890kB 16.1MB/s 
     |████████████████████████████████| 1.1MB 22.1MB/s 
     |████████████████████████████████| 3.0MB 33.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=9e07e48f908434da2b6ef3a6b7b24a3bbcbde5254f45439adbe72c21f2614e7b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
%cd /home/
import nltk
nltk.download('punkt')
from nltk import sent_tokenize, word_tokenize

filename="65_Safebuoys.txt"

from google.colab import files
uploaded = files.upload()

# print(uploaded)
# file = open(uploaded, 'rt')
# text = file.read()
# file.close()

text = str(uploaded['65_Safebuoys.txt'], 'utf-8')

txt = list(filter(lambda check: not check == [], \
                   list(map(lambda str: sent_tokenize(str), text.splitlines(True)))))

stop1 = int(len(txt) * (1./3.))
stop2 = int(len(txt) * (2./3.))


txt1 = "\n\n".join(list(map(lambda str: " ".join(str), txt[:stop1])))

sum1 = "Loocha and Corvey are confined, and Corvey lies on his makeshift bed, heavily medicated. \
He hallucinates a giant ice cream cone that is in fact a safebuoy floating in water, seen outside the small window. \
He asks Loocha what it is. This is a game they have been repeating since they were evacuated from Oort Colony. \
Corvey senses that Loocha is hiding something - flashes on the console, etc. - but she signals to let it drop and has taught him that what goes on beyond the window doesn't matter."

txt2 = "\n\n".join(list(map(lambda str: " ".join(str), txt[stop1:stop2])))

sum2="The mechanical larvae inject Corvey's body to help save his life, and Loocha gets into her cot.\
The safebuoy that Corvey thought was an ice cream cone tries to communicate with them, and Loocha debates whether to let them anchor to her safebuoy.\
She decides not to, she is afraid of what they could bring and finds safety in stability.\
Corvey awakes from the larvae, asks what is happening, and Loocha soothes him and evades his question, reminding him there is nothing to see."


txt3 = "\n\n".join(list(map(lambda str: " ".join(str), txt[stop2:])))

/home
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Saving 65_Safebuoys.txt to 65_Safebuoys (3).txt


In [ ]:
import tensorflow as tf
import json 
import os
from transformers import TFT5ForConditionalGeneration, T5Config

'''
IS_COLAB_BACKEND = 'COLAB_GPU' in os.environ  # this is always set on Colab, the value is 0 or 1 depending on GPU presence
if IS_COLAB_BACKEND:
    from google.colab import auth
    # Authenticates the Colab machine and also the TPU using your
    # credentials so that they can access your private GCS buckets.
    auth.authenticate_user()

'''

model = TFT5ForConditionalGeneration.from_pretrained('t5-large')

All model checkpoint weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-large.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained('t5')

inp = 'summarize: ' + txt1 + '<\s>' 
inp = inp.strip().replace("\n", "")

enc = tokenizer.encode_plus(inp, max_length=512, return_tensors="tf", pad_to_max_length=False)
attn_mask = enc['attention_mask']
input_ids = enc['input_ids']
#init_summary = tokenizer.tokenize(sum1.strip().replace("\n", ""), return_tensors="pt")

#print(enc)

#dec = tokenizer.decode(enc)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
#encoded = base_model(input_ids=input_ids, decoder_input_ids=input_ids)
#decoder_inputs_embeds = encoded[2]
summary_ids = model.generate(input_ids,
                              num_beams=8,
                              no_repeat_ngram_size=2,
                              min_length=30,
                              max_length=500,
                              early_stopping=True)

dec = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

outputs = model(inputs=input_ids, decoder_input_ids=input_ids, lm_labels=input_ids)
loss, prediction_scores = outputs[:2]




print(dec)


for 247 sleep cycles, the only window is a round, pillow-shaped patch of transparent steel. since the evacuation of the Oort Colony, this has been their game. "that," Loocha tells her son, is "a giant ice cream cone." he laughs, "yum." and lowers his arm and, looking sideways, smiles.


In [ ]:
import math
print(outputs)
print(math.exp(loss))

(<tf.Tensor: shape=(1, 512, 32128), dtype=float32, numpy=
array([[[-22.095272 , -12.9673195, -13.963884 , ..., -48.43187  ,
         -48.587196 , -48.428513 ],
        [-26.070868 , -12.624632 , -13.346135 , ..., -49.721962 ,
         -49.877876 , -49.994034 ],
        [-27.56967  , -15.502202 , -14.379173 , ..., -52.247066 ,
         -52.456425 , -52.615974 ],
        ...,
        [-27.399586 , -14.27532  , -15.099482 , ..., -53.29537  ,
         -53.46678  , -53.423454 ],
        [-29.618343 , -16.636827 , -17.24388  , ..., -58.83853  ,
         -58.93662  , -58.949753 ],
        [-28.135685 , -16.600739 , -14.485184 , ..., -55.352325 ,
         -55.391094 , -55.36745  ]]], dtype=float32)>, ((<tf.Tensor: shape=(1, 512, 1024), dtype=float32, numpy=
array([[[-0.02696839,  0.00989696,  0.01667405, ..., -0.10868978,
         -0.14525153, -0.18736738],
        [-0.16291729,  0.0840869 , -0.01512866, ..., -0.02864858,
          0.13856153, -0.00784154],
        [-0.03455746,  0.0955915 , -

TypeError: ignored

In [ ]:
import math

print(math.exp(loss))

941678.9999870093
